In [1]:
!nvidia-smi

Tue May  9 08:53:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !git clone https://github.com/bofenghuang/vigogne.git
# %cd vigogne
# !pip install -r requirements.txt
!pip install git+https://github.com/bofenghuang/vigogne

fatal: destination path 'vigogne' already exists and is not an empty directory.
/content/vigogne
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-fzgvb3tc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-fzgvb3tc
  Resolved https://github.com/huggingface/transformers.git to commit 006da469dd5a465f4551f4245f780e3b1e92b76c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-edto1gpv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-edto1gpv
  Resolved https://github.com/huggingface/peft.git to commit b1059b73aab9043b118ff19b0cf96263ea86248a
  Installing build dependencies .

In [3]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [5]:
# model shards of decapoda-research/llama-7b-hf are smaller, safer to use with Colab's RAM constraint
# base_model_name_or_path = "huggyllama/llama-7b"
base_model_name_or_path = "decapoda-research/llama-7b-hf"
lora_model_name_or_path = "bofenghuang/vigogne-instruct-7b"

# base_model_name_or_path = "decapoda-research/llama-13b-hf"
# lora_model_name_or_path = "bofenghuang/vigogne-instruct-13b"

# base_model_name_or_path = "bigscience/bloom-7b1"
# lora_model_name_or_path = "bofenghuang/vigogne-instruct-bloom-7b1"

load_8bit = True

tokenizer_class = LlamaTokenizer if "llama" in base_model_name_or_path else AutoTokenizer
tokenizer = tokenizer_class.from_pretrained(base_model_name_or_path, padding_side="right", use_fast=False)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model,
        lora_model_name_or_path,
        torch_dtype=torch.float16,
    )
elif device == "mps":
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        lora_model_name_or_path,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        lora_model_name_or_path,
        device_map={"": device},
    )

if not load_8bit and device != "cpu":
    model.half()  # seems to fix bugs for some users.

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
INSTRUCT_PROMPT = """Ci-dessous se trouve une instruction qui décrit une tâche à accomplir. Rédigez une réponse qui répond de manière précise à la demande.

### Instruction:
{instruction}

### Réponse:
"""

def instruct(
    instruction,
    temperature=0.1,
    do_sample=True,
    no_repeat_ngram_size=3,
    max_new_tokens=512,
    **kwargs,
):
    prompt = INSTRUCT_PROMPT.format(instruction=instruction)

    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    input_length = inputs.input_ids.shape[1]
    generated_outputs = model.generate(
        **inputs,
        generation_config=GenerationConfig(
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            no_repeat_ngram_size=no_repeat_ngram_size,
            **kwargs,
          ),
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    return tokenizer.decode(generated_tokens, skip_special_tokens=True)

In [7]:
instruct("Explique le théorème central limite.")

"Le théorême central limites stipule que la moyenne des valeurs de variables aléatoires indépendantes est égale à la moyenne de la distribution de probabilité de ces variables. Cela signifie que la distribution des valeur de la moyenne d'un ensemble de variables indépendentes est approximativement normale, même si la distribution initiale des variables n'est pas normale."

In [8]:
instruct("Répondez à la question suivante : Les pratiques artistiques transforment-elles le monde ?")

"Oui, les pratiques d'art peuvent transformer le monde en créant une expérience visuelle et émotionnelle unique. Les œuvres d'arts peuvent être utilisées pour exprimer des idées, des émotions et des messages, et peuvent avoir un impact significatif sur les gens qui les voient."

In [9]:
instruct("Traduisez le texte suivant en français : Caterpillars extract nutrients which are then converted into butterflies. People have extracted billions of nuggets of understanding and GPT-4 is humanity's butterfly.")

"Les papillons de nuit extraient des nutriments qui sont ensuite convertis en papillonne. Les gens ont extrait des milliards de noix de compréhension et GPT4 est l'oiseau de paradis de l'humanité."

In [10]:
print(instruct("Écrivez une fonction qui prend une liste de chaînes de caractères et renvoie une liste sans doublons."))

Voici une fonction Python qui prend en entrée une liste et renverra une liste avec tous les éléments de la liste, sauf les répétitions :

```
def remove_duplicates(lst):
    return list(set(lst))
```
